In [1]:
import pandas as pd
import random as rd
from itertools import combinations

class TS():
    def __init__(self, Path, seed, tabu_tenure):
        self.Path = Path
        self.seed = seed
        self.tabu_tenure = tabu_tenure
        self.instance_dict = self.input_data()
        self.Initial_solution = self.get_InitialSolution()
        self.tabu_str, self.Best_solution, self.Best_objvalue = self.TSearch()

    def input_data(self):
        return pd.read_excel(self.Path, names=['Job', 'weight', "processing_time", "due_date"],
                             index_col=0).to_dict('index')

    def get_tabuestructure(self):
        dict = {}
        for swap in combinations(self.instance_dict.keys(), 2):
            dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
        return dict

    def get_InitialSolution(self):
        n_jobs = len(self.instance_dict)  # Number of jobs
        initial_solution = list(range(1, n_jobs + 1))
        rd.seed(self.seed)
        rd.shuffle(initial_solution)
        return initial_solution

    def Objfun(self, solution):
        dict = self.instance_dict
        t = 0
        objfun_value = 0
        for job in solution:
            C_i = t + dict[job]["processing_time"]
            d_i = dict[job]["due_date"]
            T_i = max(0, C_i - d_i)
            W_i = dict[job]["weight"]
            objfun_value += W_i * T_i
            t = C_i
        return objfun_value

    def SwapMove(self, solution, i, j):
        solution = solution.copy()
        i_index = solution.index(i)
        j_index = solution.index(j)
        solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
        return solution

    def TSearch(self):
        tenure = self.tabu_tenure
        tabu_structure = self.get_tabuestructure()
        best_solution = self.Initial_solution
        best_objvalue = self.Objfun(best_solution)
        current_solution = self.Initial_solution
        current_objvalue = self.Objfun(current_solution)
        iter = 1
        Terminate = 0
        while Terminate < 100:
            if iter <= 10:
                print(f'Iteration {iter}: Best_objvalue: {best_objvalue}')
            for move in tabu_structure:
                candidate_solution = self.SwapMove(current_solution, move[0], move[1])
                candidate_objvalue = self.Objfun(candidate_solution)
                tabu_structure[move]['MoveValue'] = candidate_objvalue
            while True:
                best_move = min(tabu_structure, key=lambda x: tabu_structure[x]['MoveValue'])
                MoveValue = tabu_structure[best_move]["MoveValue"]
                tabu_time = tabu_structure[best_move]["tabu_time"]
                if tabu_time < iter:
                    current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = self.Objfun(current_solution)
                    if MoveValue < best_objvalue:
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        Terminate = 0
                    else:
                        Terminate += 1
                    tabu_structure[best_move]['tabu_time'] = iter + tenure
                    iter += 1
                    break
                else:
                    if MoveValue < best_objvalue:
                        current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                        current_objvalue = self.Objfun(current_solution)
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        Terminate = 0
                        iter += 1
                        break
                    else:
                        tabu_structure[best_move]["MoveValue"] = float('inf')
                        continue
        print("\nTabu search completed")
        print("\nPerformed iterations: {}".format(iter), "Best found Solution: {} , Objvalue: {}".format(best_solution,
                                                                                                         best_objvalue))
        return tabu_structure, best_solution, best_objvalue

print("Starting Tabu search\n")
test = TS(Path="Instance_10.xlsx", seed=2012, tabu_tenure=3)


Starting Tabu search

Iteration 1: Best_objvalue: 39.080000000000005
Iteration 2: Best_objvalue: 26.13
Iteration 3: Best_objvalue: 17.930000000000003
Iteration 4: Best_objvalue: 14.960000000000003
Iteration 5: Best_objvalue: 14.200000000000003
Iteration 6: Best_objvalue: 13.830000000000002
Iteration 7: Best_objvalue: 13.530000000000001
Iteration 8: Best_objvalue: 13.370000000000001
Iteration 9: Best_objvalue: 13.260000000000002
Iteration 10: Best_objvalue: 13.240000000000002

Tabu search completed

Performed iterations: 110 Best found Solution: [3, 2, 1, 4, 8, 10, 5, 9, 7, 6] , Objvalue: 13.240000000000002
